# 翻转棋 Reversi-v0
$4 \times 4$ 棋盘

In [1]:
import sys
import math
import itertools
import collections
import logging

import numpy as np
import pandas as pd
import gym
import tensorflow as tf
from tensorflow import keras

import boardgame2
from boardgame2 import BLACK, WHITE

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s')

### 环境
环境在 boardgame2 里实现

In [2]:
env = gym.make('Reversi-v0', board_shape=4)

### 智能体

In [3]:
def residual(x, filters, kernel_sizes=3, strides=1, activations='relu',
            regularizer=keras.regularizers.l2(1e-4)):
    shortcut = x
    for i, filte in enumerate(filters):
        kernel_size = kernel_sizes if isinstance(kernel_sizes, int) \
                else kernel_sizes[i]
        stride = strides if isinstance(strides, int) else strides[i]
        activation = activations if isinstance(activations, str) \
                else activations[i]
        z = keras.layers.Conv2D(filte, kernel_size, strides=stride,
                padding='same', kernel_regularizer=regularizer,
                bias_regularizer=regularizer)(x)
        y = keras.layers.BatchNormalization()(z)
        if i == len(filters) - 1:
            y = keras.layers.Add()([shortcut, y])
        x = keras.layers.Activation(activation)(y)
    return x

In [4]:
class AlphaZeroAgent:
    def __init__(self, env, batches=1, batch_size=4096,
            kwargs={}, load=None, sim_count=800,
            c_init=1.25, c_base=19652., prior_exploration_fraction=0.25):
        self.env = env
        self.board = np.zeros_like(env.board)
        self.batches = batches
        self.batch_size = batch_size
        
        self.net = self.build_network(**kwargs)
        self.reset_mcts()
        self.sim_count = sim_count # MCTS 次数
        self.c_init = c_init # PUCT 系数
        self.c_base = c_base # PUCT 系数
        self.prior_exploration_fraction = prior_exploration_fraction
    
    
    def build_network(self, conv_filters, residual_filters, policy_filters,
            learning_rate=0.001, regularizer=keras.regularizers.l2(1e-4)):
        # 公共部分
        inputs = keras.Input(shape=self.board.shape)
        x = keras.layers.Reshape(self.board.shape + (1,))(inputs)
        for conv_filter in conv_filters:
            z = keras.layers.Conv2D(conv_filter, 3, padding='same',
                    kernel_regularizer=regularizer,
                    bias_regularizer=regularizer)(x)
            y = keras.layers.BatchNormalization()(z)
            x = keras.layers.ReLU()(y)
        for residual_filter in residual_filters:
            x = residual(x, filters=residual_filter, regularizer=regularizer)
        intermediates = x
        
        # 概率部分
        for policy_filter in policy_filters:
            z = keras.layers.Conv2D(policy_filter, 3, padding='same',
                    kernel_regularizer=regularizer,
                    bias_regularizer=regularizer)(x)
            y = keras.layers.BatchNormalization()(z)
            x = keras.layers.ReLU()(y)
        logits = keras.layers.Conv2D(1, 3, padding='same',
                kernel_regularizer=regularizer, bias_regularizer=regularizer)(x)
        flattens = keras.layers.Flatten()(logits)
        softmaxs = keras.layers.Softmax()(flattens)
        probs = keras.layers.Reshape(self.board.shape)(softmaxs)
        
        # 价值部分
        z = keras.layers.Conv2D(1, 3, padding='same',
                kernel_regularizer=regularizer,
                bias_regularizer=regularizer)(intermediates)
        y = keras.layers.BatchNormalization()(z)
        x = keras.layers.ReLU()(y)
        flattens = keras.layers.Flatten()(x)
        vs = keras.layers.Dense(1, activation=keras.activations.tanh,
                kernel_regularizer=regularizer,
                bias_regularizer=regularizer)(flattens)
        
        model = keras.Model(inputs=inputs, outputs=[probs, vs])

        def categorical_crossentropy_2d(y_true, y_pred):
            labels = tf.reshape(y_true, [-1, self.board.size])
            preds = tf.reshape(y_pred, [-1, self.board.size])
            return keras.losses.categorical_crossentropy(labels, preds)

        loss = [categorical_crossentropy_2d, keras.losses.MSE]
        optimizer = keras.optimizers.Adam(learning_rate)
        model.compile(loss=loss, optimizer=optimizer)
        return model
    
    
    def reset_mcts(self):
        def zero_board_factory(): # 用于构造 default_dict
            return np.zeros_like(self.board, dtype=float)
        self.q = collections.defaultdict(zero_board_factory)
            # q值估计: board -> board
        self.count = collections.defaultdict(zero_board_factory)
            # q值计数: board -> board
        self.policy = {} # 策略: board -> board
        self.valid = {} # 有效位置: board -> board
        self.winner = {} # 赢家: board -> None or int
    
        
    def decide(self, observation, greedy=False, return_prob=False):
        # 计算策略
        board, player = observation
        canonical_board = player * board
        s = boardgame2.strfboard(canonical_board)
        while self.count[s].sum() < self.sim_count: # 多次 MCTS 搜索
            self.search(canonical_board, prior_noise=True)
        prob = self.count[s] / self.count[s].sum()

        # 采样
        location_index = np.random.choice(prob.size, p=prob.reshape(-1))
        location = np.unravel_index(location_index, prob.shape)
        if return_prob:
            return location, prob
        return location
    
    
    def learn(self, dfs):
        df = pd.concat(dfs).reset_index(drop=True)
        for batch in range(self.batches):
            indices = np.random.choice(len(df), size=self.batch_size)
            players, boards, probs, winners = (np.stack(
                    df.loc[indices, field]) for field in df.columns)
            canonical_boards = players[:, np.newaxis, np.newaxis] * boards
            vs = (players * winners)[:, np.newaxis]
            self.net.fit(canonical_boards, [probs, vs], verbose=0) # 训练
        self.reset_mcts()
    
    
    def search(self, board, prior_noise=False): # MCTS 搜索
        s = boardgame2.strfboard(board)
        
        if s not in self.winner:
            self.winner[s] = self.env.get_winner((board, BLACK)) # 计算赢家
        if self.winner[s] is not None: # 赢家确定的情况
            return self.winner[s]
        
        if s not in self.policy: # 未计算过策略的叶子节点
            pis, vs = self.net.predict(board[np.newaxis])
            pi, v = pis[0], vs[0]
            valid = self.env.get_valid((board, BLACK))
            masked_pi = pi * valid
            total_masked_pi = np.sum(masked_pi)
            if total_masked_pi <= 0: # 所有的有效动作都没有概率，偶尔可能发生
                masked_pi = valid # workaround
                total_masked_pi = np.sum(masked_pi)
            self.policy[s] = masked_pi / total_masked_pi
            self.valid[s] = valid
            return v
        
        # PUCT 上界计算
        count_sum = self.count[s].sum()
        coef = (self.c_init + np.log1p((1 + count_sum) / self.c_base)) * \
                math.sqrt(count_sum) / (1. + self.count[s])
        if prior_noise: # 先验噪声
            alpha = 1. / self.valid[s].sum()
            noise = np.random.gamma(alpha, 1., board.shape)
            noise *= self.valid[s]
            noise /= noise.sum()
            prior = (1. - self.prior_exploration_fraction) * \
                    self.policy[s] + \
                    self.prior_exploration_fraction * noise
        else:
            prior = self.policy[s]
        ub = np.where(self.valid[s], self.q[s] + coef * prior, np.nan)
        location_index = np.nanargmax(ub)
        location = np.unravel_index(location_index, board.shape)
        
        (next_board, next_player), _, _, _ = self.env.next_step(
                (board, BLACK), np.array(location))
        next_canonical_board = next_player * next_board
        next_v = self.search(next_canonical_board) # 递归搜索
        v = next_player * next_v
        
        self.count[s][location] += 1
        self.q[s][location] += (v - self.q[s][location]) / \
                self.count[s][location]
        return v

### 智能体和环境交互

In [5]:
def self_play(env, agent, return_trajectory=False, verbose=False):
    if return_trajectory:
        trajectory = []
    observation = env.reset()
    for step in itertools.count():
        board, player = observation
        action, prob = agent.decide(observation, return_prob=True)
        if verbose:
            print(boardgame2.strfboard(board))
            logging.info('第 {} 步：玩家 {}, 动作 {}'.format(step, player,
                    action))
        observation, winner, done, _ = env.step(action)
        if return_trajectory:
            trajectory.append((player, board, prob))
        if done:
            if verbose:
                print(boardgame2.strfboard(observation[0]))
                logging.info('赢家 {}'.format(winner))
            break
    if return_trajectory:
        df_trajectory = pd.DataFrame(trajectory,
                columns=['player', 'board', 'prob'])
        df_trajectory['winner'] = winner
        return df_trajectory
    else:
        return winner

### 训练

In [6]:
"""
AlphaZero 参数，可用来求解比较大型的问题（如五子棋）
"""
train_iterations = 700000 # 训练迭代次数
train_episodes_per_iteration = 5000 # 每次迭代自我对弈回合数
batches = 10 # 每回合进行几次批学习
batch_size = 4096 # 批学习的批大小
sim_count = 800 # MCTS需要的计数
net_kwargs = {}
net_kwargs['conv_filters'] = [256,]
net_kwargs['residual_filters'] = [[256, 256],] * 19
net_kwargs['policy_filters'] = [256,]

"""
小规模参数，用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 100
train_episodes_per_iteration = 100
batches = 2
batch_size = 64
sim_count = 200
net_kwargs = {}
net_kwargs['conv_filters'] = [256,]
net_kwargs['residual_filters'] = [[256, 256],]
net_kwargs['policy_filters'] = [256,]

agent = AlphaZeroAgent(env=env, kwargs=net_kwargs, sim_count=sim_count,
        batches=batches, batch_size=batch_size)


for iteration in range(train_iterations):
    # 自我对弈
    dfs_trajectory = []
    for episode in range(train_episodes_per_iteration):
        df_trajectory = self_play(env, agent,
                return_trajectory=True, verbose=False)
        logging.info('训练 {} 回合 {}: 收集到 {} 条经验'.format(
                iteration, episode, len(df_trajectory)))
        dfs_trajectory.append(df_trajectory)
    
    # 利用经验进行学习
    agent.learn(dfs_trajectory)
    logging.info('训练 {}: 学习完成'.format(iteration))
    
    # 演示训练结果
    self_play(env, agent, verbose=True)

2019-01-01 08:08:54,579 [INFO] 训练 0 回合 0: 收集到 12 条经验
2019-01-01 08:09:08,565 [INFO] 训练 0 回合 1: 收集到 11 条经验
2019-01-01 08:09:17,257 [INFO] 训练 0 回合 2: 收集到 12 条经验
2019-01-01 08:09:23,015 [INFO] 训练 0 回合 3: 收集到 8 条经验
2019-01-01 08:09:37,059 [INFO] 训练 0 回合 4: 收集到 12 条经验
2019-01-01 08:09:42,298 [INFO] 训练 0 回合 5: 收集到 12 条经验
2019-01-01 08:09:49,982 [INFO] 训练 0 回合 6: 收集到 12 条经验
2019-01-01 08:10:03,843 [INFO] 训练 0 回合 7: 收集到 12 条经验
2019-01-01 08:10:07,914 [INFO] 训练 0 回合 8: 收集到 6 条经验
2019-01-01 08:10:18,587 [INFO] 训练 0 回合 9: 收集到 12 条经验
2019-01-01 08:10:29,617 [INFO] 训练 0 回合 10: 收集到 12 条经验
2019-01-01 08:10:35,098 [INFO] 训练 0 回合 11: 收集到 12 条经验
2019-01-01 08:10:39,033 [INFO] 训练 0 回合 12: 收集到 8 条经验
2019-01-01 08:10:46,469 [INFO] 训练 0 回合 13: 收集到 12 条经验
2019-01-01 08:10:59,186 [INFO] 训练 0 回合 14: 收集到 9 条经验
2019-01-01 08:11:02,254 [INFO] 训练 0 回合 15: 收集到 9 条经验
2019-01-01 08:11:15,086 [INFO] 训练 0 回合 16: 收集到 12 条经验
2019-01-01 08:11:21,633 [INFO] 训练 0 回合 17: 收集到 12 条经验
2019-01-01 08:11:28,929 [INFO] 训练 0 回合 18: 

2019-01-01 08:19:14,645 [INFO] 训练 1 回合 14: 收集到 9 条经验
2019-01-01 08:19:18,405 [INFO] 训练 1 回合 15: 收集到 9 条经验
2019-01-01 08:19:25,522 [INFO] 训练 1 回合 16: 收集到 12 条经验
2019-01-01 08:19:27,357 [INFO] 训练 1 回合 17: 收集到 10 条经验
2019-01-01 08:19:32,326 [INFO] 训练 1 回合 18: 收集到 12 条经验
2019-01-01 08:19:39,076 [INFO] 训练 1 回合 19: 收集到 12 条经验
2019-01-01 08:19:42,549 [INFO] 训练 1 回合 20: 收集到 11 条经验
2019-01-01 08:19:45,731 [INFO] 训练 1 回合 21: 收集到 9 条经验
2019-01-01 08:19:54,880 [INFO] 训练 1 回合 22: 收集到 12 条经验
2019-01-01 08:20:02,810 [INFO] 训练 1 回合 23: 收集到 12 条经验
2019-01-01 08:20:03,261 [INFO] 训练 1 回合 24: 收集到 11 条经验
2019-01-01 08:20:08,973 [INFO] 训练 1 回合 25: 收集到 12 条经验
2019-01-01 08:20:13,314 [INFO] 训练 1 回合 26: 收集到 11 条经验
2019-01-01 08:20:18,235 [INFO] 训练 1 回合 27: 收集到 10 条经验
2019-01-01 08:20:22,861 [INFO] 训练 1 回合 28: 收集到 12 条经验
2019-01-01 08:20:25,478 [INFO] 训练 1 回合 29: 收集到 10 条经验
2019-01-01 08:20:30,083 [INFO] 训练 1 回合 30: 收集到 12 条经验
2019-01-01 08:20:33,975 [INFO] 训练 1 回合 31: 收集到 12 条经验
2019-01-01 08:20:33,984 [INFO] 

2019-01-01 08:25:49,321 [INFO] 训练 2 回合 50: 收集到 10 条经验
2019-01-01 08:25:56,045 [INFO] 训练 2 回合 51: 收集到 12 条经验
2019-01-01 08:25:59,201 [INFO] 训练 2 回合 52: 收集到 12 条经验
2019-01-01 08:26:02,059 [INFO] 训练 2 回合 53: 收集到 12 条经验
2019-01-01 08:26:02,319 [INFO] 训练 2 回合 54: 收集到 12 条经验
2019-01-01 08:26:04,533 [INFO] 训练 2 回合 55: 收集到 8 条经验
2019-01-01 08:26:12,871 [INFO] 训练 2 回合 56: 收集到 11 条经验
2019-01-01 08:26:18,165 [INFO] 训练 2 回合 57: 收集到 12 条经验
2019-01-01 08:26:18,499 [INFO] 训练 2 回合 58: 收集到 7 条经验
2019-01-01 08:26:18,519 [INFO] 训练 2 回合 59: 收集到 12 条经验
2019-01-01 08:26:18,999 [INFO] 训练 2 回合 60: 收集到 12 条经验
2019-01-01 08:26:19,199 [INFO] 训练 2 回合 61: 收集到 10 条经验
2019-01-01 08:26:20,641 [INFO] 训练 2 回合 62: 收集到 11 条经验
2019-01-01 08:26:20,671 [INFO] 训练 2 回合 63: 收集到 12 条经验
2019-01-01 08:26:20,681 [INFO] 训练 2 回合 64: 收集到 6 条经验
2019-01-01 08:26:20,691 [INFO] 训练 2 回合 65: 收集到 10 条经验
2019-01-01 08:26:20,711 [INFO] 训练 2 回合 66: 收集到 9 条经验
2019-01-01 08:26:23,196 [INFO] 训练 2 回合 67: 收集到 10 条经验
2019-01-01 08:26:26,640 [INFO] 训

2019-01-01 08:31:37,548 [INFO] 训练 3 回合 85: 收集到 12 条经验
2019-01-01 08:31:37,568 [INFO] 训练 3 回合 86: 收集到 12 条经验
2019-01-01 08:31:42,052 [INFO] 训练 3 回合 87: 收集到 11 条经验
2019-01-01 08:31:44,142 [INFO] 训练 3 回合 88: 收集到 12 条经验
2019-01-01 08:31:44,172 [INFO] 训练 3 回合 89: 收集到 12 条经验
2019-01-01 08:31:50,639 [INFO] 训练 3 回合 90: 收集到 12 条经验
2019-01-01 08:31:53,472 [INFO] 训练 3 回合 91: 收集到 12 条经验
2019-01-01 08:31:56,546 [INFO] 训练 3 回合 92: 收集到 12 条经验
2019-01-01 08:32:02,378 [INFO] 训练 3 回合 93: 收集到 12 条经验
2019-01-01 08:32:03,139 [INFO] 训练 3 回合 94: 收集到 12 条经验
2019-01-01 08:32:03,152 [INFO] 训练 3 回合 95: 收集到 12 条经验
2019-01-01 08:32:03,949 [INFO] 训练 3 回合 96: 收集到 10 条经验
2019-01-01 08:32:06,740 [INFO] 训练 3 回合 97: 收集到 7 条经验
2019-01-01 08:32:09,214 [INFO] 训练 3 回合 98: 收集到 12 条经验
2019-01-01 08:32:09,224 [INFO] 训练 3 回合 99: 收集到 12 条经验
2019-01-01 08:32:10,415 [INFO] 训练 3: 学习完成
++++
+ox+
+xo+
++++
2019-01-01 08:32:12,927 [INFO] 第 0 步：玩家 1, 动作 (3, 1)
++++
+ox+
+oo+
+o++
2019-01-01 08:32:15,087 [INFO] 第 1 步：玩家 -1, 动作 (3, 2)
++

2019-01-01 08:37:17,820 [INFO] 训练 5 回合 4: 收集到 10 条经验
2019-01-01 08:37:23,906 [INFO] 训练 5 回合 5: 收集到 12 条经验
2019-01-01 08:37:29,068 [INFO] 训练 5 回合 6: 收集到 10 条经验
2019-01-01 08:37:34,434 [INFO] 训练 5 回合 7: 收集到 9 条经验
2019-01-01 08:37:37,718 [INFO] 训练 5 回合 8: 收集到 12 条经验
2019-01-01 08:37:45,370 [INFO] 训练 5 回合 9: 收集到 12 条经验
2019-01-01 08:37:53,670 [INFO] 训练 5 回合 10: 收集到 11 条经验
2019-01-01 08:37:58,280 [INFO] 训练 5 回合 11: 收集到 12 条经验
2019-01-01 08:38:04,576 [INFO] 训练 5 回合 12: 收集到 9 条经验
2019-01-01 08:38:09,672 [INFO] 训练 5 回合 13: 收集到 12 条经验
2019-01-01 08:38:16,037 [INFO] 训练 5 回合 14: 收集到 12 条经验
2019-01-01 08:38:23,292 [INFO] 训练 5 回合 15: 收集到 11 条经验
2019-01-01 08:38:26,519 [INFO] 训练 5 回合 16: 收集到 12 条经验
2019-01-01 08:38:32,378 [INFO] 训练 5 回合 17: 收集到 12 条经验
2019-01-01 08:38:39,264 [INFO] 训练 5 回合 18: 收集到 12 条经验
2019-01-01 08:38:40,812 [INFO] 训练 5 回合 19: 收集到 12 条经验
2019-01-01 08:38:42,458 [INFO] 训练 5 回合 20: 收集到 12 条经验
2019-01-01 08:38:47,191 [INFO] 训练 5 回合 21: 收集到 12 条经验
2019-01-01 08:38:55,086 [INFO] 训练 5 

...

2019-01-01 14:06:44,891 [INFO] 训练 93 回合 77: 收集到 12 条经验
2019-01-01 14:06:45,852 [INFO] 训练 93 回合 78: 收集到 12 条经验
2019-01-01 14:06:47,740 [INFO] 训练 93 回合 79: 收集到 12 条经验
2019-01-01 14:06:47,749 [INFO] 训练 93 回合 80: 收集到 12 条经验
2019-01-01 14:06:47,764 [INFO] 训练 93 回合 81: 收集到 12 条经验
2019-01-01 14:06:47,773 [INFO] 训练 93 回合 82: 收集到 10 条经验
2019-01-01 14:06:49,723 [INFO] 训练 93 回合 83: 收集到 12 条经验
2019-01-01 14:06:49,741 [INFO] 训练 93 回合 84: 收集到 12 条经验
2019-01-01 14:06:50,679 [INFO] 训练 93 回合 85: 收集到 12 条经验
2019-01-01 14:06:50,856 [INFO] 训练 93 回合 86: 收集到 12 条经验
2019-01-01 14:06:51,358 [INFO] 训练 93 回合 87: 收集到 12 条经验
2019-01-01 14:06:53,255 [INFO] 训练 93 回合 88: 收集到 9 条经验
2019-01-01 14:06:53,278 [INFO] 训练 93 回合 89: 收集到 10 条经验
2019-01-01 14:06:53,288 [INFO] 训练 93 回合 90: 收集到 12 条经验
2019-01-01 14:06:53,312 [INFO] 训练 93 回合 91: 收集到 12 条经验
2019-01-01 14:06:53,334 [INFO] 训练 93 回合 92: 收集到 12 条经验
2019-01-01 14:06:53,349 [INFO] 训练 93 回合 93: 收集到 10 条经验
2019-01-01 14:06:53,369 [INFO] 训练 93 回合 94: 收集到 12 条经验
2019-01-01 

+xxx
+oxx
ooox
++++
2019-01-01 14:09:23,061 [INFO] 第 6 步：玩家 -1, 动作 (3, 0)
+xxx
+oxx
oxox
x+++
2019-01-01 14:09:23,343 [INFO] 第 7 步：玩家 1, 动作 (3, 1)
+xxx
+oxx
ooox
xo++
2019-01-01 14:09:23,343 [INFO] 第 8 步：玩家 -1, 动作 (1, 0)
+xxx
xxxx
xoox
xo++
2019-01-01 14:09:23,593 [INFO] 第 9 步：玩家 1, 动作 (0, 0)
oxxx
xoxx
xoox
xo++
2019-01-01 14:09:23,593 [INFO] 第 10 步：玩家 -1, 动作 (3, 2)
oxxx
xoxx
xxxx
xxx+
2019-01-01 14:09:23,603 [INFO] 第 11 步：玩家 1, 动作 (3, 3)
oxxx
xoxx
xxox
xxxo
2019-01-01 14:09:23,603 [INFO] 赢家 -1
2019-01-01 14:09:27,046 [INFO] 训练 95 回合 0: 收集到 12 条经验
2019-01-01 14:09:29,674 [INFO] 训练 95 回合 1: 收集到 12 条经验
2019-01-01 14:09:31,660 [INFO] 训练 95 回合 2: 收集到 12 条经验
2019-01-01 14:09:34,529 [INFO] 训练 95 回合 3: 收集到 12 条经验
2019-01-01 14:09:34,549 [INFO] 训练 95 回合 4: 收集到 12 条经验
2019-01-01 14:09:36,682 [INFO] 训练 95 回合 5: 收集到 12 条经验
2019-01-01 14:09:38,253 [INFO] 训练 95 回合 6: 收集到 12 条经验
2019-01-01 14:09:40,963 [INFO] 训练 95 回合 7: 收集到 12 条经验
2019-01-01 14:09:41,043 [INFO] 训练 95 回合 8: 收集到 12 条经验
2019-01-01 14:

2019-01-01 14:11:45,926 [INFO] 训练 96 回合 23: 收集到 12 条经验
2019-01-01 14:11:48,822 [INFO] 训练 96 回合 24: 收集到 12 条经验
2019-01-01 14:11:50,870 [INFO] 训练 96 回合 25: 收集到 12 条经验
2019-01-01 14:11:54,151 [INFO] 训练 96 回合 26: 收集到 12 条经验
2019-01-01 14:11:54,795 [INFO] 训练 96 回合 27: 收集到 12 条经验
2019-01-01 14:11:54,952 [INFO] 训练 96 回合 28: 收集到 12 条经验
2019-01-01 14:11:54,962 [INFO] 训练 96 回合 29: 收集到 12 条经验
2019-01-01 14:11:55,314 [INFO] 训练 96 回合 30: 收集到 9 条经验
2019-01-01 14:11:56,736 [INFO] 训练 96 回合 31: 收集到 12 条经验
2019-01-01 14:11:58,453 [INFO] 训练 96 回合 32: 收集到 12 条经验
2019-01-01 14:11:58,770 [INFO] 训练 96 回合 33: 收集到 12 条经验
2019-01-01 14:12:00,585 [INFO] 训练 96 回合 34: 收集到 12 条经验
2019-01-01 14:12:00,603 [INFO] 训练 96 回合 35: 收集到 12 条经验
2019-01-01 14:12:00,613 [INFO] 训练 96 回合 36: 收集到 12 条经验
2019-01-01 14:12:00,921 [INFO] 训练 96 回合 37: 收集到 12 条经验
2019-01-01 14:12:02,622 [INFO] 训练 96 回合 38: 收集到 12 条经验
2019-01-01 14:12:02,638 [INFO] 训练 96 回合 39: 收集到 10 条经验
2019-01-01 14:12:03,203 [INFO] 训练 96 回合 40: 收集到 9 条经验
2019-01-01 1

2019-01-01 14:14:12,436 [INFO] 训练 97 回合 55: 收集到 10 条经验
2019-01-01 14:14:12,446 [INFO] 训练 97 回合 56: 收集到 12 条经验
2019-01-01 14:14:13,199 [INFO] 训练 97 回合 57: 收集到 12 条经验
2019-01-01 14:14:17,618 [INFO] 训练 97 回合 58: 收集到 12 条经验
2019-01-01 14:14:17,799 [INFO] 训练 97 回合 59: 收集到 12 条经验
2019-01-01 14:14:18,050 [INFO] 训练 97 回合 60: 收集到 12 条经验
2019-01-01 14:14:18,701 [INFO] 训练 97 回合 61: 收集到 9 条经验
2019-01-01 14:14:18,711 [INFO] 训练 97 回合 62: 收集到 12 条经验
2019-01-01 14:14:19,098 [INFO] 训练 97 回合 63: 收集到 12 条经验
2019-01-01 14:14:21,680 [INFO] 训练 97 回合 64: 收集到 12 条经验
2019-01-01 14:14:21,690 [INFO] 训练 97 回合 65: 收集到 12 条经验
2019-01-01 14:14:24,870 [INFO] 训练 97 回合 66: 收集到 12 条经验
2019-01-01 14:14:24,880 [INFO] 训练 97 回合 67: 收集到 12 条经验
2019-01-01 14:14:26,359 [INFO] 训练 97 回合 68: 收集到 11 条经验
2019-01-01 14:14:26,399 [INFO] 训练 97 回合 69: 收集到 12 条经验
2019-01-01 14:14:27,725 [INFO] 训练 97 回合 70: 收集到 12 条经验
2019-01-01 14:14:27,735 [INFO] 训练 97 回合 71: 收集到 12 条经验
2019-01-01 14:14:30,571 [INFO] 训练 97 回合 72: 收集到 12 条经验
2019-01-01 

2019-01-01 14:16:34,705 [INFO] 训练 98 回合 87: 收集到 12 条经验
2019-01-01 14:16:34,995 [INFO] 训练 98 回合 88: 收集到 12 条经验
2019-01-01 14:16:35,857 [INFO] 训练 98 回合 89: 收集到 12 条经验
2019-01-01 14:16:35,867 [INFO] 训练 98 回合 90: 收集到 12 条经验
2019-01-01 14:16:35,969 [INFO] 训练 98 回合 91: 收集到 11 条经验
2019-01-01 14:16:37,519 [INFO] 训练 98 回合 92: 收集到 10 条经验
2019-01-01 14:16:37,529 [INFO] 训练 98 回合 93: 收集到 12 条经验
2019-01-01 14:16:37,547 [INFO] 训练 98 回合 94: 收集到 10 条经验
2019-01-01 14:16:37,557 [INFO] 训练 98 回合 95: 收集到 9 条经验
2019-01-01 14:16:37,835 [INFO] 训练 98 回合 96: 收集到 12 条经验
2019-01-01 14:16:37,845 [INFO] 训练 98 回合 97: 收集到 12 条经验
2019-01-01 14:16:41,654 [INFO] 训练 98 回合 98: 收集到 12 条经验
2019-01-01 14:16:44,549 [INFO] 训练 98 回合 99: 收集到 12 条经验
2019-01-01 14:16:45,487 [INFO] 训练 98: 学习完成
++++
+ox+
+xo+
++++
2019-01-01 14:16:47,545 [INFO] 第 0 步：玩家 1, 动作 (2, 0)
++++
+ox+
ooo+
++++
2019-01-01 14:16:49,050 [INFO] 第 1 步：玩家 -1, 动作 (3, 0)
++++
+ox+
oxo+
x+++
2019-01-01 14:16:49,230 [INFO] 第 2 步：玩家 1, 动作 (3, 1)
++++
+ox+
ooo+
xo++
201

### 演示训练结果

In [7]:
self_play(env, agent, verbose=True)

++++
+ox+
+xo+
++++
2019-01-01 14:18:40,029 [INFO] 第 0 步：玩家 1, 动作 (1, 3)
++++
+ooo
+xo+
++++
2019-01-01 14:18:40,029 [INFO] 第 1 步：玩家 -1, 动作 (0, 3)
+++x
+oxo
+xo+
++++
2019-01-01 14:18:40,029 [INFO] 第 2 步：玩家 1, 动作 (0, 2)
++ox
+ooo
+xo+
++++
2019-01-01 14:18:40,038 [INFO] 第 3 步：玩家 -1, 动作 (2, 3)
++ox
+oox
+xxx
++++
2019-01-01 14:18:40,038 [INFO] 第 4 步：玩家 1, 动作 (3, 0)
++ox
+oox
+oxx
o+++
2019-01-01 14:18:40,682 [INFO] 第 5 步：玩家 -1, 动作 (0, 1)
+xxx
+oxx
+oxx
o+++
2019-01-01 14:18:40,757 [INFO] 第 6 步：玩家 1, 动作 (3, 3)
+xxx
+oxx
+oox
o++o
2019-01-01 14:18:40,772 [INFO] 第 7 步：玩家 -1, 动作 (1, 0)
+xxx
xxxx
+oox
o++o
2019-01-01 14:18:40,797 [INFO] 第 8 步：玩家 1, 动作 (0, 0)
oxxx
xoxx
+oox
o++o
2019-01-01 14:18:40,807 [INFO] 第 9 步：玩家 -1, 动作 (2, 0)
oxxx
xxxx
xxxx
o++o
2019-01-01 14:18:40,811 [INFO] 赢家 -1


-1